In [1]:
import pandas as pd
import os
import shutil
from pathlib import Path
import torch
import torchaudio
import numpy as np
from scipy.io import wavfile
import pickle

In [2]:
def resample_if_necessary(signal, sr, target_sample_rate):
    """return signal"""
    if sr != target_sample_rate:
        print('resampling signal')
        resampler = torchaudio.transforms.Resample(sr, target_sample_rate)
        signal = resampler(signal)
    else:
        print('no resampling signal')
    return signal

In [3]:
def mix_down_if_necessary(signal):
    """return signal"""
    if signal.shape[0] > 1:
        print('mixing down signal')
        signal = torch.mean(signal, dim=0, keepdim=True)
    else:
        print('no mixing down signal')
    return signal

In [4]:
path_noises = 'data/NOISE/'

In [5]:
noises_folders = []
for folder_noise in os.listdir(path_noises):
    noises_files = []
    print(os.path.join(path_noises, folder_noise))
    for file_noise in os.listdir(os.path.join(path_noises, folder_noise)):
        print(os.path.join(path_noises, folder_noise, file_noise))
        noises_files.append(os.path.join(path_noises, folder_noise, file_noise))
        x, y = torchaudio.load(os.path.join(path_noises, folder_noise, file_noise))
        print(y)
        max_amplitude = x.abs().max()
        if max_amplitude <= 1.0:
            print('Audio is normalized.')
        else:
            print('Audio is not normalized')
        x = mix_down_if_necessary(x)
        x = resample_if_necessary(x, y, 16000)
        audio_norm = np.int16(x[0].numpy() / np.max(np.abs(x[0].numpy())) * 32767)
        os.remove(os.path.join(path_noises, folder_noise, file_noise))
        wavfile.write(os.path.join(path_noises, folder_noise, file_noise), 16000, audio_norm)
    noises_folders.append(noises_files)

data/NOISE/nonstationary
data/NOISE/nonstationary/babycry.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/crowd_party_adult_med.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/bell_church.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/cafeteria_babble.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/helicopter.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/people.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/nonstationary/dog_bark.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/stationary
data/NOISE/stationary/pink.wav
16000
Audio is normalized.
no mixing down signal
no resampling signal
data/NOISE/stationary/dripping_water.wav
16000
Audio is normalized.
no mi

In [6]:
noises_folders

[['data/NOISE/nonstationary/babycry.wav',
  'data/NOISE/nonstationary/crowd_party_adult_med.wav',
  'data/NOISE/nonstationary/bell_church.wav',
  'data/NOISE/nonstationary/cafeteria_babble.wav',
  'data/NOISE/nonstationary/helicopter.wav',
  'data/NOISE/nonstationary/people.wav',
  'data/NOISE/nonstationary/dog_bark.wav'],
 ['data/NOISE/stationary/pink.wav',
  'data/NOISE/stationary/dripping_water.wav',
  'data/NOISE/stationary/car.wav',
  'data/NOISE/stationary/cabin.wav',
  'data/NOISE/stationary/rain.wav',
  'data/NOISE/stationary/wind.wav',
  'data/NOISE/stationary/typing.wav']]

In [7]:
noises_source = noises_folders[0]#+noises_folders[1][0:3] #3 stationary & 3 non-stationary
noises_target = noises_folders[1]#+noises_folders[1][3:6] #2 stationary & 2 non-stationary

In [8]:
noises_source

['data/NOISE/nonstationary/babycry.wav',
 'data/NOISE/nonstationary/crowd_party_adult_med.wav',
 'data/NOISE/nonstationary/bell_church.wav',
 'data/NOISE/nonstationary/cafeteria_babble.wav',
 'data/NOISE/nonstationary/helicopter.wav',
 'data/NOISE/nonstationary/people.wav',
 'data/NOISE/nonstationary/dog_bark.wav']

In [9]:
noises_target

['data/NOISE/stationary/pink.wav',
 'data/NOISE/stationary/dripping_water.wav',
 'data/NOISE/stationary/car.wav',
 'data/NOISE/stationary/cabin.wav',
 'data/NOISE/stationary/rain.wav',
 'data/NOISE/stationary/wind.wav',
 'data/NOISE/stationary/typing.wav']

In [10]:
def generate_pkl(paths_noise, _type):
    # Guardar la variable en un archivo
    variable = paths_noise
    with open('variables/NOISES_'+str(_type)+'.pkl', 'wb') as archivo:
        pickle.dump(variable, archivo)
    # Cargar la variable desde el archivo
    with open('variables/NOISES_'+str(_type)+'.pkl', 'rb') as archivo:
        variable_cargada = pickle.load(archivo)
    # Mostrar la variable cargada
    print(variable_cargada)
    print(len(variable_cargada))

In [11]:
generate_pkl(noises_source, 'STATIONARY_NOISE_S')

['data/NOISE/nonstationary/babycry.wav', 'data/NOISE/nonstationary/crowd_party_adult_med.wav', 'data/NOISE/nonstationary/bell_church.wav', 'data/NOISE/nonstationary/cafeteria_babble.wav', 'data/NOISE/nonstationary/helicopter.wav', 'data/NOISE/nonstationary/people.wav', 'data/NOISE/nonstationary/dog_bark.wav']
7


In [12]:
generate_pkl(noises_target, 'NO_STATIONARY_NOISE_T')

['data/NOISE/stationary/pink.wav', 'data/NOISE/stationary/dripping_water.wav', 'data/NOISE/stationary/car.wav', 'data/NOISE/stationary/cabin.wav', 'data/NOISE/stationary/rain.wav', 'data/NOISE/stationary/wind.wav', 'data/NOISE/stationary/typing.wav']
7
